  # AI on the edge

Deep learning on small devices


- Motivation
- Deep learning models
- CoreML
- Tensorflow
- Movidius
- ELL

Why AI on devices?

![cloud](assets/lsvm-machine-learning-cloud.jpg)

![data center](assets/how_safe_datacenter.jpg)

![privacy](assets/GDP-Complience-V02-72dpi-RGB-01.jpg)

![qualcomm](assets\qualcomm_on_device_intelligence.png)

Deep learning models